# Multi-Armed Bandit for Data Selection

We follow in the footsteps of Gutiérrez et al (2017). Let us first formulate our objective mathematically.

#### Modelling Objective

We want to model $f: (x, \mathbf{p}) \mapsto y$, where $f$ belongs to the class of linear regression estimators parameterized by vector $\mathbf{p}$. $x_i$  and $y_i$ are the RH98 and AGBD of footprint $i$, respectively. 

Parameters $\mathbf{p}$ are estimated using a training set $S^T = \{ s_1, \ldots, s_{N_{\text{train}}} \}$ consisting of samples $s = (x,y)$. However, $S^T$ is merely a subsample from the available training data $S = \{ h_1, \ldots, h_{N_{\text{total}}} \} $ which consists of _hidden_ samples of the form $h = \{ \tilde{x}, \tilde{y}, \mathbf{m} \}$. They are hidden because RH98 $\tilde{x}$ and AGBD $\tilde{y}$ are only revealed after the sample is added to the training set. By contrast, metadata $\mathbf{m}$ about the sample is known à priori. To begin with, $S$ consists of all data in Ghana with PFT class = 2. 

#### Sample Efficiency Objective

Unlike Gutiérrez et al, we do not incur a large cost from observing $\tilde{x}$ or $\tilde{y}$. However, we are interested in selecting only the most relevant samples according to $\mathbf{m}$ all the same. We will later generalise our method to larger, more complicated datasets and more sophisticated model classes, hence sample efficiency will be important to minimise the computational burden of training the model.

We select samples by partitioning $S$ into a pre-defined number of bins $\eta_j$ for select $m_j$ $(1 \leq j \leq d)$ in $\mathbf{m}$. For instance, if $m_j$ is a categorical variable with four categories, we create the j'th partition $S = \bigcup_{k=1}^{4} C_k^j$, where $C_1^j$ contains all samples where $m_j$ is in class 1, $C_2^j$ contains all samples where $m_j$ is in class 2, etc. If $m_j$ is continuous, we quantize the variable into bins and partition accordingly. All the clusters generated using different meta information are then merged into a set of clusters $\mathcal{C} = \{C_l^j\}$. Our hypothesis is that some clusters $C_i \in \mathcal{C}$ contain more relevant information for the modelling task than others, but we do not know which. This motivates us to train a multi-armed bandit who will simultaneously _explore_ the clusters to find out which contain the most relevant information and eventually _exploit_ these clusters to maximise the share of data therefrom.

The multi-armed bandit algorithm is described in detail in their paper, but to summarise:

* At every time $t$:
    * Sample the probability of reward $\hat{\pi}_i$ from $Beta(\alpha_i, \beta_i)$ for every cluster
    * Pick a datapoint $s$ from the cluster with highest probability of reward
    * Add $s$ to $S^T$ and re-train the model
    * Predict $\hat{y}$ for a holdout validation set
    * If loss decreases (increases), $r_t = 1 \ (-1)$
    * Update $\alpha_i, \beta_i$ based on $r_t$ 

#### Summary of Objective

Our dual objectives are then:

1) To optimize the performance of $f$ at predicting $y$, as measured by the MAPE on a holdout test set.
2) To include as few samples as necessary in $S^T$.

We will train a multi-armed bandit data selector to achieve both in parallel.

### Load Packages and Data

In [50]:
import pandas as pd
import geopandas as gpd
import pyogrio
import os
import pandas.api.types as ptypes
import numpy as np
import warnings
import pickle as pkl
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from matplotlib import pyplot as plt

warnings.filterwarnings("ignore", category=DeprecationWarning) 
os.chdir(r"C:\Users\nial\OneDrive\ETH Zürich\Master Thesis")

Manually set `dtypes`

In [2]:
dtypes = {
     'pft_class': 'category'
    ,'region_cla': 'category'
    ,'leaf_off_f': 'category'
    ,'urban_prop': 'int64'
    ,'agbd': 'float64'
    ,'agbd_se': 'float64'
    ,'beam': 'category'
    ,'elev_lowes': 'float64'
    ,'lat_lowest': 'float64'
    ,'lon_lowest': 'float64'
    ,'selected_a': 'category'
    ,'shot_numbe': 'int64'
    ,'sensitivit': 'float64'
    ,'solar_elev': 'float64'
    ,'rh98': 'float64'
    ,'pattern': 'object'
    ,'doy_sin': 'float64'
    ,'doy_cos': 'float64'
    ,'date': 'int64'
    ,'lat_cos': 'float64'
    ,'lat_sin': 'float64'
    ,'lon_cos': 'float64'
    ,'lon_sin': 'float64'
    ,'pft_class_group': 'category'
    ,'geometry': 'object'
}

df_ghana_subsample = pd.read_csv('df_ghana_subsample.csv', dtype=dtypes)

Save `df_ghana`

In [3]:
df_ghana = gpd.read_file("GEDI_Ghana.geojson"
                         , driver = 'GeoJSON'
                         , engine='pyogrio')

df_ghana = df_ghana[df_ghana['pft_class'] == 2]

cols_to_keep = ['pft_class', 'leaf_off_f','urban_prop','agbd','agbd_se', 'beam', 'selected_a', 'sensitivit'\
                     , 'solar_elev', 'rh98', 'date', 'lat_cos','lat_sin','lon_cos','lon_sin', 'geometry']

df_ghana = df_ghana[cols_to_keep]

df_ghana.to_csv("df_ghana.csv", index=False)

df_ghana.head(5)

c:\Users\nial\anaconda3\envs\MasterThesis\Lib\site-packages\pyogrio\raw.py:194: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  result = ogr_read(


,pft_class,leaf_off_f,urban_prop,agbd,agbd_se,beam,selected_a,sensitivit,solar_elev,rh98,date,lat_cos,lat_sin,lon_cos,lon_sin,geometry
997,2,0,0,197.26755,17.124592,8,5,0.987684,-22.455900,24.550003,1159,0.982168,0.188008,0.998868,-0.047567,POINT (-2.72642 5.41827)
998,2,0,0,175.55383,17.139278,6,2,0.958999,-22.458270,35.879992,1159,0.982138,0.188162,0.998865,-0.047632,POINT (-2.73017 5.42277)
999,2,0,0,100.05653,17.122494,6,2,0.970559,-22.455812,21.700007,1159,0.982124,0.188234,0.998866,-0.047606,POINT (-2.72866 5.42488)
1000,2,0,0,210.39091,17.123533,6,5,0.980267,-22.444975,26.009990,1159,0.982063,0.188551,0.998872,-0.047490,POINT (-2.72202 5.43412)
1001,2,0,0,175.74130,17.122110,5,2,0.962343,-22.448800,29.920008,1159,0.982049,0.188624,0.998867,-0.047588,POINT (-2.72763 5.43625)


In [9]:
dtypes = {
     'pft_class': 'category'
    ,'region_cla': 'category'
    ,'leaf_off_f': 'category'
    ,'urban_prop': 'int64'
    ,'agbd': 'float64'
    ,'agbd_se': 'float64'
    ,'beam': 'category'
    ,'elev_lowes': 'float64'
    ,'lat_lowest': 'float64'
    ,'lon_lowest': 'float64'
    ,'selected_a': 'category'
    ,'shot_numbe': 'int64'
    ,'sensitivit': 'float64'
    ,'solar_elev': 'float64'
    ,'rh98': 'float64'
    ,'pattern': 'object'
    ,'doy_sin': 'float64'
    ,'doy_cos': 'float64'
    ,'date': 'int64'
    ,'lat_cos': 'float64'
    ,'lat_sin': 'float64'
    ,'lon_cos': 'float64'
    ,'lon_sin': 'float64'
    ,'pft_class_group': 'category'
    ,'geometry': 'object'
}

cols_to_keep = ['pft_class', 'leaf_off_f','urban_prop','agbd','agbd_se', 'beam', 'selected_a', 'sensitivit'\
                     , 'solar_elev', 'rh98', 'date', 'lat_cos','lat_sin','lon_cos','lon_sin', 'geometry']

dtypes = {key:value for key, value in dtypes.items() if key in cols_to_keep}

df_ghana = df_ghana.astype(dtypes)

### Define `bandit` class

In [39]:
class bandit:
    """
    Bandit to select data for optimizing the model f.

    INPUTS:
    dataset: includes columns x and y
    x: column name of independent variable
    y: column name of dependent variable
    features: along which to cluster df, including n_bins if numeric
    T: number of train points to sample before terminating (must be < frac_train * len(dataset))
    frac_train: fraction of dataset for training
    frac_test: fraction of dataset for testing
    frac_val: fraction of dataset for validation
    """

    def __init__(self, dataset: pd.DataFrame, x: str, y: str, features: dict, T: int=1000, batch_size: float=10\
                 , frac_train: float=0.48, frac_test: float=0.5, frac_val: float=0.02):

        self.dataset        = dataset
        self.x              = x
        self.y              = y
        self.T              = T
        self.batch_size     = batch_size
        self.frac_train     = frac_train
        self.frac_test      = frac_test
        self.frac_val       = frac_val
        self.train_indices  = []
        self.test_indices   = []
        self.val_indices    = []
        self.used_indices   = []
        self.clusters       = []
        self.model          = LinearRegression()
        self.prev_loss      = np.infty
        self.current_loss   = np.infty
        self.losses         = []
        self.rewards        = []
        self.sampled_C      = []     
        
        self.clean_clusters()
        self.generate_clusters(features)
        self.ttv_split()

        self.num_clusters   = len(self.clusters)
        self.alphas         = np.ones(self.num_clusters)
        self.betas          = np.ones(self.num_clusters)
        self.df_val         = self.dataset[self.dataset.index.isin(self.val_indices)]

    def clean_clusters(self):
        """
        Upon initialization, delete all previous cluster assignments.
        """
        cluster_cols = [c for c in self.dataset.columns if 'cluster_ID_' in c]
        self.dataset.drop(columns=cluster_cols, inplace=True)

    def generate_cluster(self, feature: str, n_bins: int=10):
        """
        Partitions the data into clusters along the specified metadata variable.
        If the feature is categorical, then the categories determine the clusters.
        If the feature is numeric, then the feature is quantized into n_bins clusters.

        INPUTS:
        feature: name of feature in dataset along which to define clusters.
        n_bins: number of bins, if feature is numeric.

        OUTPUTS:
        Adds column to self.dataset indicating the cluster ID of an observation along the given metadata feature.
        """
        # get column
        try: cluster_column = self.dataset[f'{feature}']
        except KeyError: raise KeyError(f"Column {feature} does not exist in dataset")

        # numeric
        if ptypes.is_numeric_dtype(cluster_column):

            # -1 for bins
            cluster_ids = pd.qcut(cluster_column, q=n_bins-1, labels=False, duplicates='drop')

        # categorical
        elif ptypes.is_categorical_dtype(cluster_column):
            cluster_ids = cluster_column.cat.codes

        # count
        n_cluster = len(cluster_ids.unique())

        # save IDs
        feature_name = f"cluster_ID_{feature}"
        self.dataset[feature_name] = cluster_ids

        # save clusters
        for k in range(n_cluster+1):
            self.clusters.append((feature_name, k)) 
            
    def generate_clusters(self, features: dict):
        """
        Partitions the data into clusters along the specified metadata variables.
        
        INPUTS:
        features: dictionary of form {feature: n_bins}. The n_bins argument is ignored when feature is categorical.
        """
        for feature, n_bins in features.items():
            self.generate_cluster(feature, n_bins)
        
    def ttv_split(self):
        """
        Split dataset into train, test and validation components.
        """
        data_size = len(self.dataset.index)
        assert np.isclose(self.frac_train + self.frac_test + self.frac_val, 1.0), "Train, test, validation fractions must sum to 1"

        pos_train   = int(data_size * self.frac_train)
        pos_test    = int(data_size * self.frac_test)

        split_indices = [pos_train, pos_train + pos_test]

        train_indices, test_indices, val_indices = np.split(self.dataset.index, split_indices)

        self.train_indices  = train_indices
        self.test_indices   = test_indices
        self.val_indices    = val_indices

    def sample_reward_probabilities(self):
        """
        Sample reward probabilities from multivariate Beta distribution.
        """
        pi = np.array([np.random.beta(a, b) for a, b in zip(self.alphas, self.betas)]).T
        return pi

    def sample_datapoint(self, pi):
        """
        Sample a datapoint and add to the training dataset. Return cluster sampled.

        INPUTS:
        pi: reward probabilities
        """
        # find first non-empty cluster from which to sample
        pi_descending = np.argsort(pi)[::-1]
        counter = 0

        # runs at least once and until a non-empty cluster is found
        while counter == 0 or cluster.empty:
            j = pi_descending[counter]
            feature, value = self.clusters[j]
            cluster = self.dataset[  (self.dataset[feature] == value) 
                                   & (self.dataset.index.isin(self.train_indices)) 
                                   & (~self.dataset.index.isin(self.used_indices))
                                   ]
            counter += 1
            
        # pick datapoint
        cluster_size = len(cluster.index)
        s_index = cluster.index[np.random.randint(0, cluster_size)]
        self.used_indices.append(s_index)
        self.sampled_C.append(j)
        return j
    
    def score_prediction(self, y, y_hat):
        """
        Compute loss for batch of predicitons.

        INPUTS:
        y: true values
        y_hat: predicted values
        """
        return mean_absolute_percentage_error(y, y_hat)

    def compute_reward(self):
        """
        Compute reward with current train indices.
        """
        X = self.dataset[self.dataset.index.isin(self.used_indices)][self.x].to_numpy()
        y = self.dataset[self.dataset.index.isin(self.used_indices)][self.y].to_numpy()

        # reshape for single feature
        X = X.reshape(-1, 1)

        self.model.fit(X=X, y=y)

        X_val   = self.df_val[self.x].to_numpy()
        y_val   = self.df_val[self.y].to_numpy()
        
        X_val   = X_val.reshape(-1,1)

        y_hat   = self.model.predict(X_val)

        self.prev_loss      = self.current_loss
        self.current_loss   = self.score_prediction(y=y_val, y_hat=y_hat)

        reward = 1 if self.current_loss < self.prev_loss else 0

        self.rewards.append(reward)
        self.losses.append(self.current_loss)

        return reward

    def update_beta_params(self, reward: int, j: int):
        """
        Update parameters for cluster sampling distributions based on reward.

        INPUTS:
        reward: reward from latest datapoint selection
        j: index of cluster from which datapoint selected
        """
        if reward == 1: self.alphas[j] += 1
        else:           self.betas[j] += 1

    def run_MABS(self):
        """
        Run the multi-armed bandit selection algorithm.
        """
        t = 1

        # run for T time steps
        while t <= self.T:

            # run for batch_size obs before computing reward
            for _ in range(self.batch_size):
                pi = self.sample_reward_probabilities()
                j = self.sample_datapoint(pi)

            r = self.compute_reward()
            self.update_beta_params(reward=r, j=j)

            t += 1

#### Test the agent

In [40]:
features = {'beam':None, 'sensitivit':10, 'solar_elev':10}
bandit_boi = bandit(df_ghana, x='rh98', y='agbd', features=features, T=1000, batch_size=10)

In [41]:
bandit_boi.run_MABS()

In [53]:
with open('bandit_boi.pkl', 'rb') as file:
    bandit_boi = pkl.load(file)

In [55]:
bandit_boi.clusters

[('cluster_ID_beam', 0),
 ('cluster_ID_beam', 1),
 ('cluster_ID_beam', 2),
 ('cluster_ID_beam', 3),
 ('cluster_ID_beam', 4),
 ('cluster_ID_sensitivit', 0),
 ('cluster_ID_sensitivit', 1),
 ('cluster_ID_sensitivit', 2),
 ('cluster_ID_sensitivit', 3),
 ('cluster_ID_sensitivit', 4),
 ('cluster_ID_sensitivit', 5),
 ('cluster_ID_sensitivit', 6),
 ('cluster_ID_sensitivit', 7),
 ('cluster_ID_sensitivit', 8),
 ('cluster_ID_sensitivit', 9),
 ('cluster_ID_solar_elev', 0),
 ('cluster_ID_solar_elev', 1),
 ('cluster_ID_solar_elev', 2),
 ('cluster_ID_solar_elev', 3),
 ('cluster_ID_solar_elev', 4),
 ('cluster_ID_solar_elev', 5),
 ('cluster_ID_solar_elev', 6),
 ('cluster_ID_solar_elev', 7),
 ('cluster_ID_solar_elev', 8),
 ('cluster_ID_solar_elev', 9)]